<a href="https://colab.research.google.com/github/abhisekr171-coder/data-scientist-001/blob/main/Financial_Text_Sentiment_and_Readability_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files

uploaded = files.upload()


Saving Input.xlsx to Input (2).xlsx


In [6]:
!pip install nltk pandas openpyxl
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
# 📌 STEP 1: Install necessary libraries
!pip install nltk pandas openpyxl
import nltk
nltk.download('punkt')  # For tokenizing text

# 📌 STEP 2: Import required libraries
import pandas as pd
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from google.colab import files

# 📌 STEP 3: Load Excel input
df = pd.read_excel("Input.xlsx")

# Let's display the first few rows
df.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,URL_ID,URL
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...


In [16]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Usually article text is inside <div class="td-post-content">
        content_div = soup.find('div', class_='td-post-content')

        if content_div:
            paragraphs = content_div.find_all('p')
            full_text = ' '.join([para.get_text() for para in paragraphs])
            return full_text.strip()
        else:
            return "CONTENT_NOT_FOUND"
    except:
        return "ERROR"


In [15]:
# Create a new column for extracted text
df['Text'] = df['URL'].apply(extract_text_from_url)

# Show results
df[['URL_ID', 'URL', 'Text']].head()


,URL_ID,URL,Text
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,Client: A leading IT & tech firm in the USA In...
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,Client: A leading hospital chain in the USA In...
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,Client: A leading IT & tech firm in the USA In...
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,Client: A leading automobile & tech firm in th...
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,The goal of this project is to build an Expert...


In [17]:
# Save the extracted text into a new Excel file
output_filename = "Extracted_Text.xlsx"

# Only include relevant columns
df[['URL_ID', 'URL', 'Text']].to_excel(output_filename, index=False)

# Allow download
from google.colab import files
files.download(output_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# Load the file with extracted text (if not already loaded)
df = pd.read_excel("Extracted_Text.xlsx")

# Reuse your clean_text function and dictionaries
# (Make sure clean_text, stop_words, positive_dict, negative_dict are defined)


In [24]:
def get_positive_score(tokens):
    return sum(1 for word in tokens if word in positive_dict)

def get_negative_score(tokens):
    return sum(1 for word in tokens if word in negative_dict)

def get_polarity_score(pos_score, neg_score):
    return (pos_score - neg_score) / ((pos_score + neg_score) + 1e-6)

def get_subjectivity_score(pos_score, neg_score, total_words):
    return (pos_score + neg_score) / (total_words + 1e-6)


In [27]:
# 📌 STEP 4: Define text cleaning function and dictionaries

# Define stop words (you can load a more comprehensive list)
stop_words = set([
    'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'for', 'with', 'of', 'to',
    'is', 'am', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'do', 'does', 'did', 'can', 'could', 'will', 'would', 'should', 'may', 'might',
    'must', 'shall', 'it', 'its', 'they', 'them', 'their', 'this', 'that', 'these',
    'those', 'here', 'there', 'when', 'where', 'why', 'how', 'what', 'which',
    'who', 'whom', 'whose', 'i', 'me', 'my', 'we', 'us', 'our', 'you', 'your',
    'he', 'him', 'his', 'she', 'her', 'its', 'they', 'them', 'their', 'myself',
    'himself', 'herself', 'itself', 'ourselves', 'themselves', 'each', 'every',
    'few', 'more', 'most', 'other', 'some', 'such', 'no', 'not', 'only', 'own',
    'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don',
    'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn',
    'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn',
    'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn'
])

# Define positive and negative word lists (example lists - replace with actual lists)
positive_dict = set(['good', 'great', 'excellent', 'positive', 'happy', 'joy', 'love'])
negative_dict = set(['bad', 'poor', 'terrible', 'negative', 'sad', 'pain', 'hate'])


def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words (optional, but common in sentiment analysis)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [29]:
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
import nltk

# Force re-download of all punkt-related files
nltk.download('punkt')
nltk.download('punkt_tab')  # May not be necessary, but retrying explicitly
nltk.download('averaged_perceptron_tagger')  # Sometimes needed for advanced tokenizing


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [32]:
df['Cleaned_Tokens'] = df['Text'].astype(str).apply(clean_text)


In [33]:
# Clean and tokenize text
df['Cleaned_Tokens'] = df['Text'].astype(str).apply(clean_text)

# Calculate scores
df['Positive_Score'] = df['Cleaned_Tokens'].apply(get_positive_score)
df['Negative_Score'] = df['Cleaned_Tokens'].apply(get_negative_score)
df['Polarity_Score'] = df.apply(lambda row: get_polarity_score(row['Positive_Score'], row['Negative_Score']), axis=1)
df['Subjectivity_Score'] = df.apply(lambda row: get_subjectivity_score(row['Positive_Score'], row['Negative_Score'], len(row['Cleaned_Tokens'])), axis=1)


In [34]:
output_sentiment = "Sentiment_Analysis.xlsx"
df[['URL_ID', 'Positive_Score', 'Negative_Score', 'Polarity_Score', 'Subjectivity_Score']].to_excel(output_sentiment, index=False)
files.download(output_sentiment)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
# Sentence tokenizer
def count_sentences(text):
    sentences = sent_tokenize(text)
    return len(sentences) if len(sentences) > 0 else 1

# Complex words = words with more than 2 syllables
def count_syllables(word):
    word = word.lower()
    vowels = 'aeiou'
    count = 0
    if word[0] in vowels:
        count += 1
    for i in range(1, len(word)):
        if word[i] in vowels and word[i-1] not in vowels:
            count += 1
    if word.endswith("es") or word.endswith("ed"):
        count -= 1
    if count == 0:
        count += 1
    return count

def is_complex(word):
    return count_syllables(word) > 2


In [39]:
# Word Count
df['Word_Count'] = df['Cleaned_Tokens'].apply(len)

# Sentence Count
df['Sentence_Count'] = df['Text'].astype(str).apply(count_sentences)

# Avg Sentence Length
df['Avg_Sentence_Length'] = df['Word_Count'] / df['Sentence_Count']

# Complex Word Count
df['Complex_Words'] = df['Cleaned_Tokens'].apply(lambda tokens: sum(is_complex(w) for w in tokens))

# % of Complex Words
df['Complex_Word_Percentage'] = df['Complex_Words'] / df['Word_Count']

# Fog Index
df['Fog_Index'] = 0.4 * (df['Avg_Sentence_Length'] + df['Complex_Word_Percentage'])

# Syllables per Word
df['Total_Syllables'] = df['Cleaned_Tokens'].apply(lambda tokens: sum(count_syllables(w) for w in tokens))
df['Syllables_Per_Word'] = df['Total_Syllables'] / df['Word_Count']

# Avg Word Length
df['Avg_Word_Length'] = df['Text'].astype(str).apply(lambda x: sum(len(word) for word in x.split() if word.isalpha()) / (len(x.split()) + 1e-6))


In [40]:
output_readability = "Readability_Analysis.xlsx"
df[['URL_ID', 'Avg_Sentence_Length', 'Complex_Words', 'Complex_Word_Percentage',
    'Fog_Index', 'Word_Count', 'Syllables_Per_Word', 'Avg_Word_Length']].to_excel(output_readability, index=False)

files.download(output_readability)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
import re

def count_personal_pronouns(text):
    # Convert to lowercase for matching
    text = text.lower()

    # Define regex pattern
    pattern = r'\b(i|we|my|ours|us)\b'

    # Find all matches
    pronouns = re.findall(pattern, text)

    return len(pronouns)

# Apply to your DataFrame
df['Personal_Pronouns'] = df['Text'].astype(str).apply(count_personal_pronouns)


In [42]:
# Select all final output columns
final_cols = [
    'URL_ID', 'Positive_Score', 'Negative_Score', 'Polarity_Score', 'Subjectivity_Score',
    'Avg_Sentence_Length', 'Complex_Words', 'Complex_Word_Percentage', 'Fog_Index',
    'Word_Count', 'Syllables_Per_Word', 'Avg_Word_Length', 'Personal_Pronouns'
]

# Save to Excel
final_output = "Final_Text_Analysis.xlsx"
df[final_cols].to_excel(final_output, index=False)

# Download
files.download(final_output)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>